## SQL at Scale with Spark SQL

### Creating the Database

In [0]:
%sql 
drop database if exists country_club cascade;
create database country_club;
show databases;

databaseName
country_club
default


In [0]:
# File location and type
file_location_bookings = "/FileStore/tables/Bookings.csv"
file_location_facilities = "/FileStore/tables/Facilities.csv"
file_location_members = "/FileStore/tables/Members.csv"

file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
bookings_df = (spark.read.format(file_type) 
                    .option("inferSchema", infer_schema) 
                    .option("header", first_row_is_header) 
                    .option("sep", delimiter) 
                    .load(file_location_bookings))

facilities_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_facilities))

members_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_members))

In [0]:
members_df.show(10)

+-----+--------+---------+--------------------+-------+--------------+-------------+-------------------+
memid| surname|firstname| address|zipcode| telephone|recommendedby| joindate|
+-----+--------+---------+--------------------+-------+--------------+-------------+-------------------+
 0| GUEST| GUEST| GUEST| 0|(000) 000-0000| null|2012-07-01 00:00:00|
 1| Smith| Darren|8 Bloomsbury Clos...| 4321| 555-555-5555| null|2012-07-02 12:02:05|
 2| Smith| Tracy|8 Bloomsbury Clos...| 4321| 555-555-5555| null|2012-07-02 12:08:23|
 3| Rownam| Tim|23 Highway Way, B...| 23423|(844) 693-0723| null|2012-07-03 09:32:15|
 4|Joplette| Janice|20 Crossing Road,...| 234|(833) 942-4710| 1|2012-07-03 10:25:05|
 5| Butters| Gerald|1065 Huntingdon A...| 56754|(844) 078-4130| 1|2012-07-09 10:44:09|
 6| Tracy| Burton|3 Tunisia Drive, ...| 45678|(822) 354-9973| null|2012-07-15 08:52:55|
 7| Dare| Nancy|6 Hunting Lodge W...| 10383|(833) 776-4001| 4|2012-07-25 08:59:12|
 8| Boothe| Tim|3 Bloomsbury Clos...| 234|(811) 433-2547| 3|2012-07-25 16:02:35|
 9|Stibbons| Ponder|5 Dragons Way, Wi...| 87630|(833) 160-3900| 6|2012-07-25 17:09:05|
+-----+--------+---------+--------------------+-------+--------------+-------------+-------------------+
only showing top 10 rows

### Viewing the dataframe schemas

We can take a look at the schemas of our potential tables to be written to our database soon

In [0]:
print('Bookings Schema')
bookings_df.printSchema()
print('Facilities Schema')
facilities_df.printSchema()
print('Members Schema')
members_df.printSchema()

Bookings Schema
root
-- bookid: integer (nullable = true)
-- facid: integer (nullable = true)
-- memid: integer (nullable = true)
-- starttime: string (nullable = true)
-- slots: integer (nullable = true)

Facilities Schema
root
-- facid: integer (nullable = true)
-- name: string (nullable = true)
-- membercost: double (nullable = true)
-- guestcost: double (nullable = true)
-- initialoutlay: integer (nullable = true)
-- monthlymaintenance: integer (nullable = true)

Members Schema
root
-- memid: integer (nullable = true)
-- surname: string (nullable = true)
-- firstname: string (nullable = true)
-- address: string (nullable = true)
-- zipcode: integer (nullable = true)
-- telephone: string (nullable = true)
-- recommendedby: integer (nullable = true)
-- joindate: string (nullable = true)

### Create permanent tables
We will be creating three permanent tables here in our __`country_club`__ database as we discussed previously with the following code

In [0]:
permanent_table_name_bookings = "country_club.Bookings"
bookings_df.write.format("parquet").saveAsTable(permanent_table_name_bookings)

permanent_table_name_facilities = "country_club.Facilities"
facilities_df.write.format("parquet").saveAsTable(permanent_table_name_facilities)

permanent_table_name_members = "country_club.Members"
members_df.write.format("parquet").saveAsTable(permanent_table_name_members)

### Refresh tables and check them

In [0]:
%sql
use country_club;
REFRESH table bookings;
REFRESH table facilities;
REFRESH table members;
show tables;

database,tableName,isTemporary
country_club,bookings,false
country_club,facilities,false
country_club,members,false


### Test a sample SQL query

__Note:__ You can use __`%sql`__ at the beginning of a cell and write SQL queries directly as seen in the following cell. Neat isn't it!

In [0]:
%sql
select * from bookings limit 3

bookid,facid,memid,starttime,slots
0,3,1,2012-07-03 11:00:00,2
1,4,1,2012-07-03 08:00:00,2
2,6,0,2012-07-03 18:00:00,2


#### Q1: Some of the facilities charge a fee to members, but some do not. Please list the names of the facilities that do.

In [0]:
%sql
SELECT DISTINCT name, membercost
FROM facilities
WHERE membercost > 0

name,membercost
Massage Room 2,9.9
Massage Room 1,9.9
Squash Court,3.5
Tennis Court 2,5.0
Tennis Court 1,5.0


####  Q2: How many facilities do not charge a fee to members?

In [0]:
%sql
SELECT COUNT(DISTINCT name) as free_membership_count
FROM facilities
WHERE membercost = 0

free_membership_count
4


#### Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost? 
#### Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [0]:
%sql
SELECT facid, 
        name, 
        membercost, 
        monthlymaintenance
FROM facilities
WHERE membercost > 0
AND (membercost < monthlymaintenance*0.2)

facid,name,membercost,monthlymaintenance
0,Tennis Court 1,5.0,200
1,Tennis Court 2,5.0,200
4,Massage Room 1,9.9,3000
5,Massage Room 2,9.9,3000
6,Squash Court,3.5,80


#### Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.

In [0]:
%sql
SELECT *
FROM facilities
WHERE facid IN (1,5)

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
1,Tennis Court 2,5.0,25.0,8000,200
5,Massage Room 2,9.9,80.0,4000,3000


#### Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100? 
#### Return the name and monthly maintenance of the facilities in question.

In [0]:
%sql
SELECT name, monthlymaintenance,
CASE WHEN monthlymaintenance > 100
  THEN 'expensive'
  ELSE 'cheap'
END as label
FROM facilities

name,monthlymaintenance,label
Tennis Court 1,200,expensive
Tennis Court 2,200,expensive
Badminton Court,50,cheap
Table Tennis,10,cheap
Massage Room 1,3000,expensive
Massage Room 2,3000,expensive
Squash Court,80,cheap
Snooker Table,15,cheap
Pool Table,15,cheap


#### Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution.

In [0]:
%sql
SELECT firstname, surname
FROM members
WHERE joindate = (SELECT MAX(joindate) FROM members)

firstname,surname
Darren,Smith


####  Q7: How can you produce a list of all members who have used a tennis court?
- Include in your output the name of the court, and the name of the member formatted as a single column. 
- Ensure no duplicate data
- Also order by the member name.

In [0]:
%sql
SELECT DISTINCT CONCAT(firstname, ' ', surname) AS member, 
                name as facility
FROM members
INNER JOIN bookings
  ON members.memid = bookings.memid
INNER JOIN facilities
  ON bookings.facid = facilities.facid
WHERE name LIKE '%Tennis Court%'

member,facility
Charles Owen,Tennis Court 2
Joan Coplin,Tennis Court 1
David Pinker,Tennis Court 1
David Jones,Tennis Court 2
Tracy Smith,Tennis Court 2
David Farrell,Tennis Court 2
Charles Owen,Tennis Court 1
GUEST GUEST,Tennis Court 2
Florence Bader,Tennis Court 1
Millicent Purview,Tennis Court 2


#### Q8: How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? 

- Remember that guests have different costs to members (the listed costs are per half-hour 'slot')
- The guest user's ID is always 0. 

#### Include in your output the name of the facility, the name of the member formatted as a single column, and the cost.

- Order by descending cost, and do not use any subqueries.

In [0]:
%sql
SELECT name AS facility,
        CONCAT(firstname, ' ', surname) AS member, 
        CASE WHEN members.memid = 0
            THEN guestcost * slots
            ELSE membercost * slots
        END AS cost
FROM bookings
INNER JOIN members
  ON bookings.memid = members.memid
INNER JOIN facilities
  ON bookings.facid = facilities.facid
WHERE starttime >=  '2012-09-14'
  AND starttime < '2012-09-15'
  AND CASE WHEN members.memid = 0
    THEN guestcost * slots
    ELSE membercost * slots
  END > 30
ORDER BY cost DESC

facility,member,cost
Massage Room 2,GUEST GUEST,320.0
Massage Room 1,GUEST GUEST,160.0
Massage Room 1,GUEST GUEST,160.0
Massage Room 1,GUEST GUEST,160.0
Tennis Court 2,GUEST GUEST,150.0
Tennis Court 2,GUEST GUEST,75.0
Tennis Court 1,GUEST GUEST,75.0
Tennis Court 1,GUEST GUEST,75.0
Squash Court,GUEST GUEST,70.0
Massage Room 1,Jemima Farrell,39.6


#### Q9: This time, produce the same result as in Q8, but using a subquery.

In [0]:
%sql
SELECT name AS facility,
        CONCAT(firstname, ' ', surname) AS member,
        cost
FROM
    (SELECT firstname,
            surname,
            name,
            starttime,
            CASE WHEN members.memid = 0
              THEN guestcost * slots
              ELSE membercost * slots
            END AS cost
      FROM bookings
      INNER JOIN members
         ON bookings.memid = members.memid
      INNER JOIN facilities
         ON bookings.facid = facilities.facid) AS subquery
WHERE starttime >=  '2012-09-14'
  AND starttime < '2012-09-15'
  AND cost > 30
ORDER BY cost DESC

facility,member,cost
Massage Room 2,GUEST GUEST,320.0
Massage Room 1,GUEST GUEST,160.0
Massage Room 1,GUEST GUEST,160.0
Massage Room 1,GUEST GUEST,160.0
Tennis Court 2,GUEST GUEST,150.0
Tennis Court 2,GUEST GUEST,75.0
Tennis Court 1,GUEST GUEST,75.0
Tennis Court 1,GUEST GUEST,75.0
Squash Court,GUEST GUEST,70.0
Massage Room 1,Jemima Farrell,39.6


#### Q10: Produce a list of facilities with a total revenue less than 1000.
- The output should have facility name and total revenue, sorted by revenue. 
- Remember that there's a different cost for guests and members!

In [0]:
%sql
SELECT name,
       SUM(CASE WHEN memid = 0
             THEN guestcost * slots
             ELSE membercost * slots
           END) AS total_revenue
FROM bookings
INNER JOIN facilities
  ON bookings.facid = facilities.facid
GROUP BY name

name,total_revenue
Pool Table,270.0
Tennis Court 2,14310.0
Squash Court,13468.0
Tennis Court 1,13860.0
Badminton Court,1906.5
Massage Room 1,50351.600000000275
Massage Room 2,14454.599999999995
Table Tennis,180.0
Snooker Table,240.0


In [0]:
%sql
SELECT name,
       total_revenue
FROM 
    (SELECT name,
            SUM(CASE WHEN memid = 0
                   THEN guestcost * slots
                   ELSE membercost * slots
                 END) AS total_revenue
     FROM bookings
     INNER JOIN facilities
       ON bookings.facid = facilities.facid
     GROUP BY name) as subquery
WHERE total_revenue < 1000
ORDER BY total_revenue
        

name,total_revenue
Table Tennis,180.0
Snooker Table,240.0
Pool Table,270.0
